In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.core.defchararray import add as npStrAdd
import ipysheet
from ipywidgets import Button, HBox, VBox

tdata = pd.read_pickle('Transaction.pkl')
bdata = pd.read_pickle('Budget.pkl')
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

rows = 15
columns = 5

In [17]:
def createSheet(tdata, startIndex, rows, columns, key="A"):
    S = ipysheet.sheet(rows = rows, columns = columns, key=key)
    Cells = np.zeros((rows, columns)).tolist()
    for i in range(rows):
        if i + startIndex < tdata.shape[0]:
            Cells[i][0] = ipysheet.cell(i, 0, tdata.iloc[i + startIndex, 0].strftime('%d/%m/%Y'))
            Cells[i][1] = ipysheet.cell(i, 1, tdata.iloc[i + startIndex, 1].astype(str))
            Cells[i][2] = ipysheet.cell(i, 2, tdata.iloc[i + startIndex, 2])
            if tdata.iloc[i + startIndex, 3] == '' or not np.isfinite(tdata.iloc[i + startIndex, 3]):
                Cells[i][3] = ipysheet.cell(i, 3, '')
            else:
                Cells[i][3] = ipysheet.cell(i, 3, tdata.iloc[i + startIndex, 3])

            Cells[i][4] = ipysheet.cell(i, 4, tdata.iloc[i + startIndex, 4])
        else:
            Cells[i][0] = ipysheet.cell(i, 0, '')
            Cells[i][1] = ipysheet.cell(i, 1, '')
            Cells[i][2] = ipysheet.cell(i, 2, '')
            Cells[i][3] = ipysheet.cell(i, 3, '')
            Cells[i][4] = ipysheet.cell(i, 4, '')
            
    return S, Cells
    
def readSheet(tdata, startIndex, rows, columns, S, Cells):
    for i in range(rows):
        if i + startIndex < tdata.shape[0]:
            tdata.iloc[i + startIndex, :] = pd.Series(
                [np.datetime64(pd.to_datetime(Cells[i][0].value, dayfirst=True), 's')] + 
                [float(Cells[i][1].value)] + 
                [Cells[i][2].value, Cells[i][3].value, Cells[i][4].value], 
                index=tdata.columns)
        else:
            if Cells[i][0].value != '':
                row = pd.Series(
                [np.datetime64(pd.to_datetime(Cells[i][0].value, dayfirst=True), 's')] + 
                [float(Cells[i][1].value)] + 
                [Cells[i][2].value, Cells[i][3].value, Cells[i][4].value], 
                index=tdata.columns)

                tdata = tdata.append(row, ignore_index=True)
        
    return tdata

def updateSheet(tdata, startIndex, rows, columns, S, Cells):
    for i in range(rows):
        if i + startIndex < tdata.shape[0]:
            Cells[i][0].value = tdata.iloc[i + startIndex, 0].strftime('%d/%m/%Y')
            Cells[i][1].value = tdata.iloc[i + startIndex, 1].astype(str)
            Cells[i][2].value = tdata.iloc[i + startIndex, 2]
            # print(tdata.iloc[i + startIndex, 3], type(tdata.iloc[i + startIndex, 3]))
            if not isinstance(tdata.iloc[i + startIndex, 3], str) and (tdata.iloc[i + startIndex, 3] == '' or not np.isfinite(tdata.iloc[i + startIndex, 3])):
                Cells[i][3].value = ''
            else:
                Cells[i][3].value = tdata.iloc[i + startIndex, 3]

            Cells[i][4].value = tdata.iloc[i + startIndex, 4]
        else:
            Cells[i][0].value = ''
            Cells[i][1].value = ''
            Cells[i][2].value = ''
            Cells[i][3].value = ''
            Cells[i][4].value = ''
            
    return S, Cells

In [21]:
startIndex = tdata.shape[0] - 5

In [4]:
tdata.iloc[startIndex:, :]

,Date,Amt,Category,TAG,Summary
332,2019-10-10,99.00,Shopping,NaN,Jacket
333,2019-10-10,10.60,Eat-Out,NaN,Messina
334,2019-10-12,79.79,Utilities,NaN,Arc
335,2019-10-12,29.85,Groceries,NaN,Coles
336,2019-10-12,13.55,Groceries,NaN,Meat


In [5]:
S, Cells = createSheet(tdata, startIndex, rows, columns)
S

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='10/10/2019'), Cell…

In [40]:
tdata = readSheet(tdata, startIndex, rows, columns, S, Cells)

In [41]:
tdata.iloc[startIndex:, :]

,Date,Amt,Category,TAG,Summary
391,2019-11-04,52.96,Shopping,,Tripod Stand
392,2019-11-06,8.50,Eat-Out,,Sushi
393,2019-11-10,200.00,Cash,,Cash
394,2019-11-07,80.50,Eat-Out,,Rashays
395,2019-11-08,15.00,Eat-Out,,GyG
396,2019-11-08,1.00,Eat-Out,,7-Eleven
397,2019-11-08,68.05,Groceries,,Coles
398,2019-11-09,5.70,Eat-Out,,Hungry Jacks
399,2019-11-09,1.10,Eat-Out,,Coles Express


In [39]:
startIndex += 13
S, Cells = updateSheet(tdata, startIndex, rows, columns, S, Cells)

In [43]:
# WARNING!! Make certain!
if False:
    print('WARNING: Writing Out!!')
    tdata.to_pickle('Transaction.pkl')

## Buttons

In [11]:
words = ['correct', 'horse', 'battery', 'staple']
items = [Button(description=w) for w in words]

msg = ''

def gen(text):
    def clicked(*args, **kwargs):
        global msg
        msg = '{} called with {}, {}'.format(text, args, kwargs)
    
    return clicked

def msg ():
    global msg
    msg = 'sdsd'

for i, btn in enumerate(items):
    btn.on_click(gen(words[i]))
    

left_box = VBox([items[0], items[1]])
right_box = VBox([items[2], items[3]])
HBox([left_box, right_box])

In [13]:
msg

"battery called with (Button(description='battery', style=ButtonStyle()),), {}"